# Data Augmented Question Answering

This notebook uses some generic prompts/language models to evaluate an question answering system that uses other sources of data besides what is in the model. For example, this can be used to evaluate a question answering system over your propritary data.

## Setup
Let's set up an example with our favorite example - the state of the union address.

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI, VectorDBQA

In [2]:
with open('../state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)
qa = VectorDBQA.from_llm(llm=OpenAI(), vectorstore=docsearch)

## Examples
Now we need some examples to evaluate. We can do this in two ways:

1. Hard code some examples ourselves
2. Generate examples automatically, using a language model

In [3]:
# Hard-coded examples
examples = [
    {
        "query": "What did the president say about Ketanji Brown Jackson",
        "answer": "He praised her legal ability and said he nominated her for the supreme court."
    },
    {
        "query": "What did the president say about Michael Jackson",
        "answer": "Nothing"
    }
]

In [4]:
# Generated examples
from langchain.evaluation.qa import QAGenerateChain
example_gen_chain = QAGenerateChain.from_llm(OpenAI())

In [5]:
new_examples = example_gen_chain.apply_and_parse([{"doc": t} for t in texts[:5]])

In [6]:
new_examples

[{'query': "Who did Russia's Vladimir Putin miscalculate when he attempted to shake the foundations of the free world?",
  'answer': 'Putin miscalculated the Ukrainian people.'},
 {'query': 'What did President Zelenskyy say in his speech to the European Parliament?',
  'answer': '"Light will win over darkness."'},
 {'query': 'What countries joined the coalition to confront Putin?',
  'answer': 'France, Germany, Italy, the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland.'},
 {'query': 'What measures is the US taking to punish Russia and its oligarchs?',
  'answer': "The US is enforcing powerful economic sanctions, cutting off Russia's largest banks from the international financial system, preventing the Russian Ruble from being defended, choking off Russia's access to technology, and joining with European allies to seize yachts, luxury apartments, and private jets."},
 {'query': 'What is the total amount of direct assistance being provided

In [7]:
# Combine examples
examples += new_examples

## Evaluate
Now that we have examples, we can use the question answering evaluator to evaluate our question answering chain.

In [8]:
from langchain.evaluation.qa import QAEvalChain

In [9]:
predictions = qa.apply(examples)

In [10]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)

In [11]:
graded_outputs = eval_chain.evaluate(examples, predictions)

In [14]:
for i, eg in enumerate(examples):
    print(f"Example {i}:")
    print("Question: " + predictions[i]['query'])
    print("Real Answer: " + predictions[i]['answer'])
    print("Predicted Answer: " + predictions[i]['result'])
    print("Predicted Grade: " + graded_outputs[i]['text'])
    print()

Example 0:
Question: What did the president say about Ketanji Brown Jackson
Real Answer: He praised her legal ability and said he nominated her for the supreme court.
Predicted Answer:  The president said that Ketanji Brown Jackson is one of the nation's top legal minds and is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He said she is a consensus builder and has received a broad range of support since being nominated.
Predicted Grade:  CORRECT

Example 1:
Question: What did the president say about Michael Jackson
Real Answer: Nothing
Predicted Answer:  The president did not mention Michael Jackson.
Predicted Grade:  CORRECT

Example 2:
Question: Who did Russia's Vladimir Putin miscalculate when he attempted to shake the foundations of the free world?
Real Answer: Putin miscalculated the Ukrainian people.
Predicted Answer:  Putin miscalculated the strength of the Ukrainian people, the sup